In [1]:
import numpy as np
import itertools
import ast
from scipy.linalg import expm


In [43]:
# ----- CONFIG -----
beta = 1.0  # inverse temperature
input_file = "test_H.txt"

# ----- FERMIONIC BASIS -----
def generate_half_filling_basis(n_orbitals):
    n_particles = n_orbitals // 2
    basis = []
    for state in itertools.combinations(range(n_orbitals), n_particles):
        vec = [0] * n_orbitals
        for idx in state:
            vec[idx] = 1
        basis.append(tuple(vec))
    return basis

def state_index(state, basis):
    return basis.index(tuple(state))

# ----- OPERATOR LOGIC -----
def apply_op(op_string, state):
    new_state = list(state)
    phase = 1

    for op in op_string:
        dagger = op.endswith('^')
        idx = int(op[:-1] if dagger else op)

        if dagger:  # creation
            if new_state[idx] == 1:
                return None, 0  # annihilated
            phase *= (-1) ** sum(new_state[:idx])
            new_state[idx] = 1
        else:  # annihilation
            if new_state[idx] == 0:
                return None, 0
            phase *= (-1) ** sum(new_state[:idx])
            new_state[idx] = 0

    return tuple(new_state), phase


def parse_term(line):
    line = line.strip().rstrip('+').strip()  # Remove trailing '+' and whitespace
    coeff_str, ops_str = line.split('[')
    coeff = complex(coeff_str.strip())
    ops = ops_str.strip(' ]').split()
    return coeff, ops


# ----- MAIN HAMILTONIAN BUILDING -----
def build_hamiltonian_matrix(n_orbitals, lines):
    basis = generate_half_filling_basis(n_orbitals)
    dim = len(basis)
    H = np.zeros((dim, dim), dtype=complex)

    for line in lines:
        coeff, ops = parse_term(line)

        ops = ops[::-1]

        print(ops)
        for i, state in enumerate(basis):
            new_state, phase = apply_op(ops, state)
            if new_state:
                j = state_index(new_state, basis)
                H[i, j] += coeff * phase

        # Hermitian conjugate term
        hc_ops = [op[:-1] if op.endswith('^') else op + '^' for op in ops[::-1]]
        hc_coeff = np.conj(coeff)

        for i, state in enumerate(basis):
            new_state, phase = apply_op(hc_ops, state)
            if new_state:
                j = state_index(new_state, basis)
                if i != j:  # Avoid double-counting diagonal terms
                    H[i, j] += hc_coeff * phase

    return H, basis

# ----- EXPECTATION VALUE -----
def compute_thermal_expectation(H, beta):
    expH = expm(-beta * H)
    Z = np.trace(expH)
    return np.trace(H @ expH).real / Z.real


In [44]:
with open(input_file, 'r') as f:
    lines = [line for line in f if line.strip() and not line.startswith('#')]

# Determine the number of orbitals (max index + 1)
all_ops = [parse_term(line)[1] for line in lines]
max_orb = max(int(op[:-1] if op.endswith('^') else op) for ops in all_ops for op in ops) + 1 # Assuming lowest orbital starts from 0

H, basis = build_hamiltonian_matrix(max_orb, lines)
print("Hamiltonian matrix:")
print(H)

exp_val = compute_thermal_expectation(H, beta)
print(f"\nThermal expectation value of H (beta={beta}): {exp_val:.6f}")

['2^']


ValueError: (1, 1, 1, 0) is not in list

In [42]:
basis = generate_half_filling_basis(4)
print(basis[0])
newstate , phase = apply_op(['2^' , '1'] , basis[0])
print(newstate , phase)

(1, 1, 0, 0)
(1, 0, 1, 0) -1
